In [1]:
# https://github.com/TomAugspurger/effective-pandas/blob/master/modern_1_intro.ipynb

import os
import zipfile

import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import requests

headers = {
    'Referer': 'https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time',
    'Origin': 'https://www.transtats.bts.gov',
    'Content-Type': 'application/x-www-form-urlencoded',
}

params = (
    ('Table_ID', '236'),
    ('Has_Group', '3'),
    ('Is_Zipped', '0'),
)

location1 = r'C:\Users\adai\Documents\GitHub\effective-pandas\modern-1-url.txt'
with open(location1, encoding='utf-8') as f:
    data = f.read().strip()

os.makedirs('data', exist_ok=True)
dest = "data/flights.csv.zip"

if not os.path.exists(dest):
    r = requests.post('https://www.transtats.bts.gov/DownLoad_Table.asp',
                      headers=headers, params=params, data=data, stream=True)

    with open("data/flights.csv.zip", 'wb') as f:
        for chunk in r.iter_content(chunk_size=102400): 
            if chunk:
                f.write(chunk)

In [3]:
zf = zipfile.ZipFile("data/flights.csv.zip")
fp = zf.extract(zf.filelist[0].filename, path='data/')
df = pd.read_csv(fp, parse_dates=["FL_DATE"]).rename(columns=str.lower)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450017 entries, 0 to 450016
Data columns (total 33 columns):
fl_date                  450017 non-null datetime64[ns]
unique_carrier           450017 non-null object
airline_id               450017 non-null int64
tail_num                 449378 non-null object
fl_num                   450017 non-null int64
origin_airport_id        450017 non-null int64
origin_airport_seq_id    450017 non-null int64
origin_city_market_id    450017 non-null int64
origin                   450017 non-null object
origin_city_name         450017 non-null object
dest_airport_id          450017 non-null int64
dest_airport_seq_id      450017 non-null int64
dest_city_market_id      450017 non-null int64
dest                     450017 non-null object
dest_city_name           450017 non-null object
crs_dep_time             450017 non-null int64
dep_time                 441476 non-null float64
dep_delay                441476 non-null float64
taxi_out                

In [4]:
df.ix[10:15, ['fl_date', 'tail_num']]

,fl_date,tail_num
10,2017-01-01,N560AA
11,2017-01-01,N169AA
12,2017-01-01,N915AA
13,2017-01-01,N915AA
14,2017-01-01,N826AA
15,2017-01-01,N678AA


In [5]:
first = df.groupby('airline_id')[['fl_date', 'unique_carrier']].first()
first.head()

,fl_date,unique_carrier
airline_id,,
19393,2017-01-01,WN
19690,2017-01-01,HA
19790,2017-01-01,DL
19805,2017-01-01,AA
19930,2017-01-01,AS


In [6]:
first.ix[10:15, ['fl_date', 'tail_num']]

,fl_date,tail_num
airline_id,,


In [7]:
first = df.groupby('unique_carrier').first()
first.ix[10:15, ['fl_date', 'tail_num']]

,fl_date,tail_num
unique_carrier,,
VX,2017-01-01,N846VA
WN,2017-01-01,N8510E


In [8]:
first.loc[['AA', 'AS', 'DL'], ['fl_date', 'tail_num']]

,fl_date,tail_num
unique_carrier,,
AA,2017-01-01,N837AA
AS,2017-01-01,N557AS
DL,2017-01-01,N942DL


In [9]:
first.iloc[[0, 1, 3], [0, 1]]

,fl_date,airline_id
unique_carrier,,
AA,2017-01-01,19805
AS,2017-01-01,19930
DL,2017-01-01,19790


In [10]:
f = pd.DataFrame({'a':[1,2,3,4,5], 'b':[10,20,30,40,50]})
f

,a,b
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [11]:
# Ignore context manager for now
with pd.option_context('mode.chained_assignment', None):
    f[f['a'] <= 3]['b'] = f[f['a'] <= 3]['b'] / 10
f

,a,b
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [ ]:
f.loc[f['a'] <= 3, 'b'] = f.loc[f['a'] <= 3, 'b'] / 10
f

,a,b
0,1,1.0
1,2,2.0
2,3,3.0
3,4,40.0
4,5,50.0


In [ ]:
hdf = df.set_index(['unique_carrier', 'origin', 'dest', 'tail_num', 'fl_date']).sort_index()
hdf[hdf.columns[:4]].head()

In [ ]:
hdf.loc[['AA', 'DL', 'US'], ['dep_time', 'dep_delay']]

In [ ]:
hdf.loc[(['AA', 'DL', 'US'], ['ORD', 'DSM']), ['dep_time', 'dep_delay']]

In [ ]:
hdf.loc[pd.IndexSlice[:, ['ORD', 'DSM']], ['dep_time', 'dep_delay']]

In [ ]:
pd.IndexSlice[:, ['ORD', 'DSM']]

In [ ]:
hdf.loc[pd.IndexSlice[:, 'ORD', 'DSM', :, '2017-01-01':'2017-01-15'],
       ['dep_time', 'dep_delay', 'arr_time', 'arr_delay']]